In [1]:
# Import source

cd("../Source")
include("ChiralMC.jl")
using ChiralMC

include("functions_msc.jl")
include("functions_observables.jl")
include("functions_parallel.jl")

using Base.Test
using Plots
gr()
using Distributions
cd("../Notebooks/")

In [7]:
println("\nTesting Energy difference function ΔE\n----------------------------------------------------------------")
# Test case. We make a random 5x5 lattice, and put two different lattice sites in the middle
# i.e. the [2,2] position. Then we use the different functions to calculate the energy
# difference associated with this change.

L = 5
ψ₂ = State(2,L)
site = LatticeSite() # Get random lattice site
ψ₁ = copy(ψ₂)
@test ψ₁.nb[3,2].ϕᵣ₊₂.u⁺ == ψ₁.lattice[2,2].u⁺
@test ψ₁.nb[3,2].ϕᵣ₊₂ == ψ₁.lattice[2,2]
set!(ψ₁.lattice[2,2], site)
@test ψ₁.nb[3,2].ϕᵣ₊₂.u⁺ == site.u⁺

dE = ΔE(site,ψ₂.lattice[2,2],ψ₂.nb[2,2],ψ₂.nnb[2,2],ψ₂.nnnb[2,2], 2,ψ₂.consts)
println("Checking that ΔE and E get the same result")
println("ΔE\t\t\tE₁-E₂")
println("$dE\t$(E(ψ₁)-E(ψ₂))")
println(dE/(E(ψ₁)-E(ψ₂)))
println(@test isapprox(E(ψ₁)-E(ψ₂), dE; atol=0, rtol=1e-13))


Testing Energy difference function ΔE
----------------------------------------------------------------
Checking that ΔE and E get the same result
ΔE			E₁-E₂
-5.371279643836078e6	-5.371279643836141e6
0.9999999999999883
Test Passed


In [8]:

println("Checking that ΔE gives same result as E'-E on all different sites of the lattice")
println("[v_pos, h_pos]\t|ΔE-(E₁-E₂)|")
for h_pos = 1:L, v_pos = 1:L
    ψ₁ = copy(ψ₂)
    ϕ′ = LatticeSite()
    dE = ΔE(ϕ′, ψ₂.lattice[v_pos,h_pos],ψ₂.nb[v_pos,h_pos],ψ₂.nnb[v_pos,h_pos],ψ₂.nnnb[v_pos,h_pos],h_pos,ψ₂.consts)
    set!(ψ₁.lattice[v_pos,h_pos], ϕ′)
    println("[$(v_pos), $h_pos]\t\t$(abs(dE-(E(ψ₁)-E(ψ₂))))")
    @test isapprox(E(ψ₁)-E(ψ₂), dE; atol=0, rtol=5e-13)
end
println(@test isapprox(E(ψ₁)-E(ψ₂), dE; atol=0, rtol=5e-13))


Checking that ΔE gives same result as E'-E on all different sites of the lattice
[v_pos, h_pos]	|ΔE-(E₁-E₂)|
[1, 1]		4.470348358154297e-8
[2, 1]		5.21540641784668e-8
[3, 1]		5.587935447692871e-8
[4, 1]		1.4156103134155273e-7
[5, 1]		5.960464477539063e-8
[1, 2]		2.9802322387695312e-8
[2, 2]		4.0046870708465576e-8
[3, 2]		2.9802322387695312e-8
[4, 2]		3.3527612686157227e-8
[5, 2]		4.842877388000488e-8
[1, 3]		7.450580596923828e-9
[2, 3]		5.960464477539063e-8
[3, 3]		1.1175870895385742e-7
[4, 3]		1.4901161193847656e-8
[5, 3]		6.705522537231445e-8
[1, 4]		1.4901161193847656e-8
[2, 4]		1.862645149230957e-8
[3, 4]		1.862645149230957e-8
[4, 4]		3.166496753692627e-8
[5, 4]		2.421438694000244e-8
[1, 5]		4.842877388000488e-8
[2, 5]		4.470348358154297e-8
[3, 5]		1.564621925354004e-7
[4, 5]		1.2665987014770508e-7
[5, 5]		1.1548399925231934e-7
Test Passed


In [3]:
########################################################################################
#                     Testing Symmetric Energy functions
#
#######################################################################################

println("\nTesting Energy function E(ψ)\n----------------------------------------------------------------")
println("Test case: uᵣ⁺=const., uᵣ⁻=const., θ⁺=θ⁻=Aᵣ,ᵢ=0")
println("Testing mostly Fₖ and Fᵥ")

L = 100
f = 0.0
γ = 1.0
ν = rand()

# First we create a state with all phases and fluctuating gauge fields 0.
c = SystConstants(L, γ, 1.0, ν, f, 0.5)
ψ = State(1, c)
u⁺ = rand()
u⁻ = rand()
for h_pos = 1:L, v_pos = 1:L
    ψ.lattice[v_pos,h_pos].u⁺ = u⁺
    ψ.lattice[v_pos,h_pos].u⁻ = u⁻
end

# Calculate theoretical energy
teoretisk = γ^4*L^2*(u⁺^2*u⁻^2*(2+ν)+u⁺^4/2+u⁻^4/2) - L^2*(u⁺^2+u⁻^2)
println("teoretisk =\t$(teoretisk)")
println("E =\t\t$(E(ψ))")
println(@test isapprox(E(ψ), teoretisk, atol=0, rtol=1e-13*L^2))


Testing Energy function E(ψ)
----------------------------------------------------------------
Test case: uᵣ⁺=const., uᵣ⁻=const., θ⁺=θ⁻=Aᵣ,ᵢ=0
Testing mostly Fₖ and Fᵥ
teoretisk =	-0.6414910474159115
E =		-0.6414910474150659
Test Passed


In [4]:
println("\nTest case: uᵣ⁺=const., uᵣ⁻=const., θ⁺=θ⁻=0, Aᵣ,ᵢ=δᵢ₂⋅2πf⋅rₓ")
println("Tests the use of non-fluctuating vector potential")

L = 100
f = 1/L*rand(1:L-1)
γ = 1.0
ν = rand()

c = SystConstants(L, γ, 1.0, ν, f, 0.5)
ψ = State(1, c)
u⁺ = rand()
u⁻ = rand()
for h_pos = 1:L, v_pos = 1:L
    ψ.lattice[v_pos,h_pos].u⁺ = u⁺
    ψ.lattice[v_pos,h_pos].u⁻ = u⁻
end

χ = sum([cos(4π*f*rₓ) for rₓ=0:(L-1)])

teoretisk = -0.5*(u⁺^2 + u⁻^2)*(L^2 + L*χ) + (ν+1)/2*L*u⁺*u⁻*(χ - L) + L^2*(u⁺^2*u⁻^2*(2+ν)+u⁺^4/2+u⁻^4/2)
println("teoretisk =\t$(teoretisk)")
println("E =\t\t$(E(ψ))")
println(@test isapprox(E(ψ), teoretisk, atol=0, rtol=1e-13*L^2))


Test case: uᵣ⁺=const., uᵣ⁻=const., θ⁺=θ⁻=0, Aᵣ,ᵢ=δᵢ₂⋅2πf⋅rₓ
Tests the use of non-fluctuating vector potential
teoretisk =	-1181.4925440171028
E =		-1181.4925440171146
Test Passed


In [5]:
println("\nTest case: uᵣ⁺=const., uᵣ⁻=const., θ⁺=θ⁻=0, Aᵣ,ᵢ=δᵢ₂⋅2πf⋅rₓ, except at r=r₀ where θᵣ⁺=θ⁺, θᵣ⁻=θ⁻")
println("Tests full higgs energy function with MGT term")

L = 100
f = 1/L*rand(1:L-1)
γ = 1.0
ν = rand()*0.9

c = SystConstants(L, γ, 1.0, ν, f, 0.5)
ψ = State(1, c)
u⁺ = rand()
u⁻ = rand()
for h_pos = 1:L, v_pos = 1:L
    ψ.lattice[v_pos,h_pos].u⁺ = u⁺
    ψ.lattice[v_pos,h_pos].u⁻ = u⁻
end

# Pick r₀ in the middle and set phases
r₀ = [floor(Int, L/2), floor(Int, L/2)]
θ⁺ = mod2pi(rand()*two_pi)
ψ.lattice[r₀...].θ⁺ = θ⁺
θ⁻ = mod2pi(rand()*two_pi)
ψ.lattice[r₀...].θ⁻ = θ⁻

# Calculate theoretical energy
rₓ = r₀[2]-1
χ = L*sum([cos(4π*f*x) for x=0:(L-1)]) -2*cos(4π*f*rₓ)
A = 2π*f
Fₘ = (ν-1)/4*u⁺*u⁻*(sin(θ⁺-A*(rₓ-1)) - sin(θ⁺-A*(rₓ+1)) + sin(θ⁺+A*(rₓ+1)) - sin(θ⁺+A*(rₓ-1)) 
    - (sin(θ⁻-A*(rₓ-1)) - sin(θ⁻-A*(rₓ+1)) + sin(θ⁻+A*(rₓ+1)) - sin(θ⁻+A*(rₓ-1))))
Fₖ = -0.5*((u⁺^2+u⁻^2)*(L^2-2+χ) + 2*(u⁺^2*cos(θ⁺)+u⁻^2*cos(θ⁻)) 
    + u⁺^2*cos(θ⁺+4π*f*rₓ)+u⁻^2*cos(θ⁻+4π*f*rₓ)+u⁺^2*cos(θ⁺-4π*f*rₓ)+u⁻^2*cos(θ⁻-4π*f*rₓ))
Fₐₙ = (ν+1)/4*u⁺*u⁻*(2*χ - 2*(L^2-2) - 2*(cos(θ⁺)+cos(θ⁻))
    + cos(θ⁺-4π*f*rₓ) + cos(θ⁺+4π*f*rₓ) + cos(θ⁻-4π*f*rₓ) + cos(θ⁻+4π*f*rₓ))
Fᵥ = (u⁺*u⁻)^2*((L^2-1)*(2+ν) + 2 + ν*cos(2*(θ⁺-θ⁻))) + L^2/2*(u⁺^4+u⁻^4)
teoretisk = Fₘ + Fₖ + Fₐₙ + Fᵥ

# Compare with energy function
println("teoretisk =\t$(teoretisk)")
println("E =\t\t$(E(ψ))")
println(@test isapprox(E(ψ), teoretisk, atol=0, rtol=1e-13))


Test case: uᵣ⁺=const., uᵣ⁻=const., θ⁺=θ⁻=0, Aᵣ,ᵢ=δᵢ₂⋅2πf⋅rₓ, except at r=r₀ where θᵣ⁺=θ⁺, θᵣ⁻=θ⁻
Tests full higgs energy function with MGT term
teoretisk =	-47.850345956643835
E =		-47.85034595664161
Test Passed
